# Numerical and statistical tools

Based on a notebook by Chase Coleman and Spencer Lyon and on material from QuantEcon

18 December 2017

This notebook covers a selection of additional packages, covering topics in statistics, data handling, and other areas of interest.

Anyone interested in autodiff can look at [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) and [ReverseDiff.jl](https://github.com/JuliaDiff/ReverseDiff.jl)

## Distributions.jl

Provides routines for working with probability distributions and...
    - Computing moments/statistics: mean, median, mode, entropy, mgf, quantile
    - Probability evaluation: pdf, cdf, ccdf, quantile, invlogcdf
    - Sampling: rand and rand!

[Documentation](http://distributionsjl.readthedocs.io/en/latest/)

In [27]:
# Pkg.add("Distributions")

### Distributions.jl Basics

In [1]:
using Distributions

In [29]:
# all subtypes of `Distributions.Distribution`
length(subtypes(Distribution))

67

In [2]:
?Normal  # good documentation

search: Normal normalize normalize! NormalCanon normalize_string



```
Normal(μ,σ)
```

The *Normal distribution* with mean `μ` and standard deviation `σ` has probability density function

$$
f(x; \mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}}
\exp \left( - \frac{(x - \mu)^2}{2 \sigma^2} \right)
$$

```julia
Normal()          # standard Normal distribution with zero mean and unit variance
Normal(mu)        # Normal distribution with mean mu and unit variance
Normal(mu, sig)   # Normal distribution with mean mu and variance sig^2

params(d)         # Get the parameters, i.e. (mu, sig)
mean(d)           # Get the mean, i.e. mu
std(d)            # Get the standard deviation, i.e. sig
```

External links

  * [Normal distribution on Wikipedia](http://en.wikipedia.org/wiki/Normal_distribution)


In [3]:
dists = [
    Normal(0, 1),
    Beta(1.0, 2.0),
    Chisq(5),
    Frechet(5.0, 2.0),
    Gamma(1.0, 2.0),
    Pareto(3.0, 2.0),
    Binomial(10, 0.6),
    Poisson(0.7),
    MvLogNormal(ones(2), 3*eye(2)),
    Dirichlet([0.1, 0.2, 0.3, 0.4]),
    InverseWishart(5, eye(2)),
    MixtureModel(Normal[
        Normal(-2.0, 1.2),
        Normal(0.0, 1.0),
        Normal(3.0, 2.5)], 
        [0.2, 0.5, 0.3]  # prior
    )
]

for d in dists
    println("Working with distribution: $(repr(d))")
    @show mean(d)
    if isa(d, Distributions.UnivariateDistribution)
        @show rand(d, 2, 2)
    else
        @show rand(d, 2)
    end
    
    @show pdf(d, rand(d))
    println("\n\n\n")
end

Working with distribution: Distributions.Normal{Float64}(μ=0.0, σ=1.0)
mean(d) = 0.0
rand(d, 2, 2) = [1.13714 -0.183199; -0.552657 -0.821711]
pdf(d, rand(d)) = 0.3181295845549784




Working with distribution: Distributions.Beta{Float64}(α=1.0, β=2.0)
mean(d) = 0.3333333333333333
rand(d, 2, 2) = [0.170901 0.0361851; 0.204202 0.739539]
pdf(d, rand(d)) = 1.3037006671046503




Working with distribution: Distributions.Chisq{Float64}(ν=5.0)
mean(d) = 5.0
rand(d, 2, 2) = [5.50546 4.6375; 12.0028 3.93241]
pdf(d, rand(d)) = 0.13638158972826442




Working with distribution: Distributions.Frechet{Float64}(α=5.0, θ=2.0)
mean(d) = 2.3284594274506065
rand(d, 2, 2) = [1.75435 1.72247; 1.66146 1.67428]
pdf(d, rand(d)) = 0.8873754496107239




Working with distribution: Distributions.Gamma{Float64}(α=1.0, θ=2.0)
mean(d) = 2.0
rand(d, 2, 2) = [0.0447799 0.431371; 1.7511 2.16835]
pdf(d, rand(d)) = 0.002599582677349569




Working with distribution: Distributions.Pareto{Float64}(α=3.0, θ=2.0)
mean(d) =

### More than you need


Let's list all the available distributions, by type of distribution

In [32]:
dist_types = [
    Distributions.DiscreteMatrixDistribution,
    Distributions.DiscreteMultivariateDistribution,
    Distributions.DiscreteUnivariateDistribution,
    Distributions.ContinuousMatrixDistribution,
    Distributions.ContinuousMultivariateDistribution,
    Distributions.ContinuousUnivariateDistribution,   
]

for T in dist_types
    println("$T: ")
    @show subtypes(T)
    println("\n\n")
end 

Distributions.Distribution{Distributions.Matrixvariate,Distributions.Discrete}: 
subtypes(T) = Union{DataType, UnionAll}[Distributions.AbstractMixtureModel{Distributions.Matrixvariate,Distributions.Discrete,C} where C<:Distributions.Distribution]



Distributions.Distribution{Distributions.Multivariate,Distributions.Discrete}: 
subtypes(T) = Union{DataType, UnionAll}[Distributions.AbstractMixtureModel{Distributions.Multivariate,Distributions.Discrete,C} where C<:Distributions.Distribution, Distributions.DirichletMultinomial, Distributions.Multinomial]



Distributions.Distribution{Distributions.Univariate,Distributions.Discrete}: 
subtypes(T) = Union{DataType, UnionAll}[Distributions.AbstractMixtureModel{Distributions.Univariate,Distributions.Discrete,C} where C<:Distributions.Distribution, Distributions.Bernoulli, Distributions.BetaBinomial, Distributions.Binomial, Distributions.Categorical, Distributions.DiscreteUniform, Distributions.Geometric, Distributions.Hypergeometric, Distribu

In [33]:
# fitting a distribution, given some samples
fit_mle(Normal, randn(100_000)) # should get close to N(0, 1)

Distributions.Normal{Float64}(μ=-0.0015332949351311543, σ=1.0000359500569347)

In [34]:
# do fitting with mle
fit_mle(Uniform, rand(100_000) .* 2 .+ 1) # should get close to U(1, 3)

Distributions.Uniform{Float64}(a=1.0000075805099509, b=2.999950042341385)

## Calculus.jl

- Computes analytical derivatives of Julia `Expr`essions and accurate numerical derivatives of functions

[Package](https://github.com/johnmyleswhite/Calculus.jl)

In [35]:
# Pkg.add("Calculus")

### Calculus.jl Basics

In [4]:
using Calculus

#### Symbolic derivatives

In [5]:
differentiate(:(sin(x)), :x)

:(cos(x))

In [38]:
differentiate(:(cos(sin(y))), :y)

:(cos(y) * -(sin(sin(y))))

In [6]:
differentiate(:(c^(1-γ)/(1-γ)), :c)

:(((1 - γ) * c ^ ((1 - γ) - 1)) / (1 - γ))

#### Finite difference

In [40]:
derivative(sin, 1.0) - cos(1.0)

-5.036193684304635e-12

In [41]:
second_derivative(sin, 1.0) + sin(1.0)

-6.647716624952338e-7

In [7]:
Calculus.gradient(x -> exp(x[1]) + sin(x[2]) / x[1], [1.0, π])

2-element Array{Float64,1}:
  2.71828
 -1.0    

In [43]:
Calculus.hessian(x -> exp(x[1]) + sin(x[2]) / x[1], [1.0, π])

2×2 Array{Float64,2}:
 2.71828  1.0       
 1.0      1.71123e-7

In [44]:
Calculus.jacobian(x -> [exp(x[1]),  sin(x[2]) / x[1]], [1.0, π], :central)

2×2 Array{Float64,2}:
  2.71828       0.0
 -1.22465e-16  -1.0

## SymEngine.jl

- Next generation C++ backend for sympy computer algebra system
- A very fast alternative to Calculus.jl for symbolic differentiation

[Jula package](https://github.com/symengine/SymEngine.jl) and ["Documentation"](https://github.com/symengine/symengine/blob/master/symengine/cwrapper.h)

In [45]:
# Pkg.add("SymEngine")

### SymEngine.jl Basics

In [46]:
using SymEngine

In [47]:
# needs first argument to be of type SymEngine.Basic
diff(Basic(:(sin(x))), :x)

cos(x)

In [48]:
diff(Basic("cos(sin(y))"), :y)

-cos(y)*sin(sin(y))

In [49]:
diff(Basic("c^(1-γ)/(1-γ)"), :c)

c^(-γ)

Let's see how fast SymEngine is compared to Calculus.jl

To do this we will load the BenchmarkTools.jl package that goes to great lengths to produce statistically accurate and robust timing estimates at the sub-microsecond level

In [50]:
# Pkg.add("BenchmarkTools")
using BenchmarkTools

In [51]:
@benchmark Calculus.differentiate(:((y + r*a - ap)^(1-γ)/(1-γ)), :ap)

BenchmarkTools.Trial: 
  memory estimate:  66.66 KiB
  allocs estimate:  1109
  --------------
  minimum time:     827.381 μs (0.00% GC)
  median time:      877.233 μs (0.00% GC)
  mean time:        1.218 ms (1.21% GC)
  maximum time:     11.096 ms (88.43% GC)
  --------------
  samples:          4075
  evals/sample:     1

In [52]:
@benchmark diff(Basic("(y + r*a - ap)^(1-γ)/(1-γ)"), :ap)

BenchmarkTools.Trial: 
  memory estimate:  840 bytes
  allocs estimate:  42
  --------------
  minimum time:     56.437 μs (0.00% GC)
  median time:      58.088 μs (0.00% GC)
  mean time:        59.610 μs (0.00% GC)
  maximum time:     374.913 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Data handling

We won't cover these packages in detail, but the following are some of the most commonly used data-related packages available today. 

- [DataFrames.jl](https://github.com/JuliaStats/DataFrames.jl): Provides a DataFrame type for handling columnar data. Similar to data frames in Python Pandas or R.
- [CSV.jl](https://github.com/JuliaData/CSV.jl): reading and writing of delimited data files
- [DataStreams.jl](https://github.com/JuliaData/DataStreams.jl): provide an interface for streaming data from a source to a sink
- [Query.jl](https://github.com/davidanthoff/Query.jl): filter, project, join, group any iterable data source
- [HDF5.jl](https://github.com/JuliaIO/HDF5.jl): Read and write HDF5 files, commonly used by MATLAB and others to store numeric data
- [JLD.jl](https://github.com/JuliaIO/JLD.jl): Read and write datasets using Julia-native JLD files, which are enhanced HDF5 files 
